In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
!pip install pyyaml==5.4.1
!pip install darts
import darts
print(darts.__version__)

!pip install -U optuna==2.0.0
# !pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
      /tmp/pip-build-env-m53r1ha9/overlay/lib/python3.10/site-packages/setuptools/config/setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ********************************************************************************
              The license_file parameter is deprecated, use license_files instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ************************

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0.26.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.9/226.9 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 12.6 MB/s eta 0:00:00
  Created wheel for optuna: filename=optuna-2.0.0-py3-none-any.whl size=312840 sha256=83aa19f538334fa3b330d6fc526c7fb93d142165333397fa6869f91513e3be0d
  Stored in directory: /root/.cache/pip/wheels/88/08/60/552399d1ae00b0d56554e4c0f80809c4c42a94de19e68ceed3
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=4ccfda2d8fd8c4f872d9f5b7951272a32dcf271499663c4bd3764be5b24ac37e
  Stored in directory: /root/.cache/pip/wheels/04/24/fe/140a94a7f1036003ede945

In [3]:
import numpy as np
import time

from darts import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries
from darts.models import LightGBMModel, CatBoostModel, Prophet, RNNModel, TFTModel, NaiveSeasonal, ExponentialSmoothing, NHiTSModel
from darts.metrics import mape, smape, rmse, rmsle
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist
from darts.utils.likelihood_models import QuantileRegression
from darts.utils.missing_values import fill_missing_values
from darts.models.filtering.moving_average_filter import MovingAverageFilter

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from tqdm import tqdm

import sklearn
from sklearn import preprocessing

import pandas as pd
import torch
import matplotlib.pyplot as plt
import gc
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
torch.manual_seed(1126)
np.random.seed(1126)

In [5]:
# Load all Datasets
df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
df_test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
df_holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
df_transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
df_sample_submission = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')

# Sales Data (Target)

family_list = df_train['family'].unique()
family_list
store_list = df_stores['store_nbr'].unique()
store_list

train_merged = pd.merge(df_train, df_stores, on ='store_nbr')
train_merged = train_merged.sort_values(["store_nbr","family","date"])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})
df_test_dropped = df_test.drop(['onpromotion'], axis=1)
df_test_sorted = df_test_dropped.sort_values(by=['store_nbr','family'])

In [6]:
family_TS_dict = {}

for family in family_list:
  df_family = train_merged.loc[train_merged['family'] == family]

  list_of_TS_family = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
                                value_cols="sales", # target variable
                                fill_missing_dates=True,
                                freq='D')
  for ts in list_of_TS_family:
            ts = ts.astype(np.float32)

  list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
  family_TS_dict[family] = list_of_TS_family

# Transform the Sales Data

family_pipeline_dict = {}
family_TS_transformed_dict = {}

for key in family_TS_dict:
  train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
  log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
  train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
  training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
  family_pipeline_dict[key] = train_pipeline
  family_TS_transformed_dict[key] = training_transformed

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
list_of_TS = TimeSeries.from_group_dataframe(
                                train_merged,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
                                value_cols="sales", # target variable
                                fill_missing_dates=True,
                                freq='D')
for ts in list_of_TS:
            ts = ts.astype(np.float32)

list_of_TS = sorted(list_of_TS, key=lambda ts: int(ts.static_covariates_values()[0,0]))

# Transform the Sales Data

train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")
train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
training_transformed = train_pipeline.fit_transform(list_of_TS)

#train_merged.head()

# Create 7-day and 28-day moving average of sales

sales_moving_average_7 = MovingAverageFilter(window=7)
sales_moving_average_28 = MovingAverageFilter(window=28)

sales_moving_averages_dict = {}

for key in family_TS_transformed_dict:
  sales_mas_family = []
  
  for ts in family_TS_transformed_dict[key]:
    ma_7 = sales_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="sales_ma_7")
    ma_28 = sales_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="sales_ma_28")
    mas = ma_7.stack(ma_28)
    sales_mas_family.append(mas)
  
  sales_moving_averages_dict[key] = sales_mas_family

In [8]:
# General Covariates (Time-Based and Oil)

full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')

# Time-Based Covariates

year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

# Transform
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

In [9]:
# Oil Price

oil = TimeSeries.from_dataframe(df_oil, 
                                time_col = 'date', 
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverageFilter(window=7)
oil_moving_average_28 = MovingAverageFilter(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

# Stack General Covariates Together

general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)


In [10]:
# Store-Specific Covariates (Transactions and Holidays)

# Transactions
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())   # necessary workaround to remove static covariates (so I can stack covariates later on)
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'), 
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

# Moving Averages for Transactions
trans_moving_average_7 = MovingAverageFilter(window=7)
trans_moving_average_28 = MovingAverageFilter(window=28)

transactions_covs = []

for ts in transactions_transformed:
  ma_7 = trans_moving_average_7.filter(ts).astype(np.float32)
  ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="transactions_ma_7")
  ma_28 = trans_moving_average_28.filter(ts).astype(np.float32)
  ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="transactions_ma_28")
  trans_and_mas = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions").stack(ma_7).stack(ma_28)
  transactions_covs.append(trans_and_mas)

In [11]:
# Re-Defining Categories of Holidays in a Meaningful Way

df_holidays_events['type'] = np.where(df_holidays_events['transferred'] == True,'Transferred', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Transfer','Holiday', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Additional','Holiday', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Bridge','Holiday', 
                                      df_holidays_events['type'])


# Assign Holidays to all TimeSeries and Save in Dictionary

def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):
            
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
            
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):
            
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

def holiday_TS_list_54(holiday_list):

    listofseries = []
    
    for i in range(0,54):
            
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries

list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)   
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

In [12]:
store_covariates_future = []

for store in range(0,len(store_list)):
    stacked_covariates = holidays_transformed[store].stack(general_covariates)  
    store_covariates_future.append(stacked_covariates)

store_covariates_past = []
holidays_transformed_sliced = holidays_transformed # for slicing past covariates

for store in range(0,len(store_list)):
    holidays_transformed_sliced[store] = holidays_transformed[store].slice_intersect(transactions_covs[store])
    general_covariates_sliced = general_covariates.slice_intersect(transactions_covs[store])
    stacked_covariates = transactions_covs[store].stack(holidays_transformed_sliced[store]).stack(general_covariates_sliced)  
    store_covariates_past.append(stacked_covariates)
    
# Store/Family-Varying Covariates (Promotion)

df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in family_list:
    df_family = df_promotion.loc[df_promotion['family'] == family]

    list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="onpromotion", # covariate of interest
                                fill_missing_dates=True,
                                freq='D')
  
    for ts in list_of_TS_promo:
        ts = ts.astype(np.float32)

    family_promotion_dict[family] = list_of_TS_promo

promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
    promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])
  
    promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])

    # Moving Averages for Promotion Family Dictionaries
    promo_moving_average_7 = MovingAverageFilter(window=7)
    promo_moving_average_28 = MovingAverageFilter(window=28)

    promotion_covs = []

    for ts in promotion_transformed:
        ma_7 = promo_moving_average_7.filter(ts)
        ma_7 = TimeSeries.from_series(ma_7.pd_series())  
        ma_7 = ma_7.astype(np.float32)
        ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
        ma_28 = promo_moving_average_28.filter(ts)
        ma_28 = TimeSeries.from_series(ma_28.pd_series())  
        ma_28 = ma_28.astype(np.float32)
        ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
        promo_and_mas = ts.stack(ma_7).stack(ma_28)
        promotion_covs.append(promo_and_mas)

    promotion_transformed_dict[key] = promotion_covs

# 2.5. Assemble All Covariates in Dictionaries

past_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

    promotion_family = promotion_transformed_dict[key]
    sales_mas = sales_moving_averages_dict[key]
    covariates_past = [promotion_family[i].slice_intersect(store_covariates_past[i]).stack(store_covariates_past[i].stack(sales_mas[i])) for i in range(0,len(promotion_family))]

    past_covariates_dict[key] = covariates_past

future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

    promotion_family = promotion_transformed_dict[key]
    covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

    future_covariates_dict[key] = covariates_future

only_past_covariates_dict = {}

for key in tqdm(sales_moving_averages_dict):
    sales_moving_averages = sales_moving_averages_dict[key]
    only_past_covariates = [sales_moving_averages[i].stack(transactions_covs[i]) for i in range(0,len(sales_moving_averages))]

    only_past_covariates_dict[key] = only_past_covariates

100%|██████████| 33/33 [00:04<00:00,  7.96it/s]


In [13]:
del(df_train)
del(df_test)
del(df_stores)
del(df_holidays_events)
del(df_oil)
del(df_transactions)
gc.collect()

22

In [14]:
# Data Preparation for N-HiTS

def flatten(l):
    return [item for sublist in l for item in sublist]

future_covariates_full = []

for family in family_list:
    future_covariates_full.append(future_covariates_dict[family])
    
future_covariates_full = flatten(future_covariates_full)

# Shift future covariates back so they can be used as past covariates

only_past_covariates = []

for family in family_list:
    only_past_covariates.append(only_past_covariates_dict[family])
    
only_past_covariates = flatten(only_past_covariates)

NHiTS_covariates = []

for i in range(0,len(future_covariates_full)):
    shifted = future_covariates_full[i].shift(n=-16)
    cut = shifted.slice_intersect(only_past_covariates[i])
    stacked = cut.stack(only_past_covariates[i])
    NHiTS_covariates.append(stacked)
    
# Split in train/val/test for Tuning and Validation

val_len = 16

train = [s[: -(2 * val_len)] for s in training_transformed]
val = [s[-(2 * val_len) : -val_len] for s in training_transformed]
test = [s[-val_len:] for s in training_transformed]

In [15]:
def build_fit_nhits_model(
    input_chunk_length,
    num_stacks,
    num_blocks,
    num_layers,
    layer_exp,
    dropout,
    lr,
    likelihood = None,
    callbacks = None
):
    
    MAX_N_EPOCHS = 50
    MAX_SAMPLES_PER_TS = 180
    
    early_stopper = EarlyStopping(monitor = "val_loss", min_delta = 1e-4, 
                                  patience = 2, verbose = True)
    
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks
    
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0], 
            "callbacks": callbacks,
        }
    
    model = NHiTSModel(input_chunk_length = input_chunk_length, output_chunk_length = 16,
                       num_stacks = num_stacks, num_blocks = num_blocks, 
                       num_layers = num_layers, layer_widths = 2 ** layer_exp,
                       dropout = dropout, n_epochs = MAX_N_EPOCHS, batch_size = 128,
                       add_encoders = None, likelihood = None, loss_fn = torch.nn.MSELoss(),
                       random_state = 1126, optimizer_kwargs = {"lr": lr},
                       pl_trainer_kwargs=pl_trainer_kwargs, model_name = "nhits",
                       force_reset = True, save_checkpoints = True)     
    model_val_set = [s[-((2 * val_len) + input_chunk_length) : -val_len] for s in training_transformed]
    
    model.fit(series = train, val_series = model_val_set, past_covariates = NHiTS_covariates,
              val_past_covariates = NHiTS_covariates,  max_samples_per_ts = MAX_SAMPLES_PER_TS)
    
    model = NHiTSModel.load_from_checkpoint("nhits")
    
    return model

In [16]:
def objective(trail):
    
    callback = [PyTorchLightningPruningCallback(trail, monitor = "val_loss")]
    
    input_chunk_length = trail.suggest_int("input_chunk_length", 63, 270)
    num_stacks = trail.suggest_int("num_stacks", 1, 3)
    num_blocks = trail.suggest_int("num_blocks", 1, 3)
    num_layers = trail.suggest_int("num_layers", 1, 3)
    layer_exp = trail.suggest_int("layer_exp", 7, 10)
    dropout = trail.suggest_float("dropout", 0.01, 0.2, step = 0.01)
    lr = trail.suggest_float("lr", 5e-5, 0.1, log = True)
    
    model = build_fit_nhits_model(input_chunk_length, num_stacks, num_blocks, num_layers, 
                                  layer_exp, dropout, lr, callbacks = callback)
    
    y_preds = model.predict(series = train, past_covariates = NHiTS_covariates, 
                            n = val_len)
    val_loss = rmsle(val, y_preds, verbose = True)
    val_loss = np.mean(val_loss)
    
    return val_loss if val_loss != np.nan else float("inf")

def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")
    
torch.cuda.empty_cache()

study_nhits = optuna.create_study()
study_nhits.optimize(objective, n_trials = 5, callbacks = [print_callback])

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

  0%|          | 0/1782 [00:00<?, ?it/s]

[I 2023-11-11 17:06:20,132] Trial 4 finished with value: 0.18164051340797338 and parameters: {'input_chunk_length': 194, 'num_stacks': 1, 'num_blocks': 1, 'num_layers': 1, 'layer_exp': 8, 'dropout': 0.09, 'lr': 0.02765282074985536}. Best is trial 2 with value: 0.06079723007900688.


Current value: 0.18164051340797338, Current params: {'input_chunk_length': 194, 'num_stacks': 1, 'num_blocks': 1, 'num_layers': 1, 'layer_exp': 8, 'dropout': 0.09, 'lr': 0.02765282074985536}
Best value: 0.06079723007900688, Best params: {'input_chunk_length': 164, 'num_stacks': 2, 'num_blocks': 3, 'num_layers': 2, 'layer_exp': 7, 'dropout': 0.060000000000000005, 'lr': 0.00026200382993823117}


In [17]:
print(f"Best value: {study_nhits.best_value}, Best params: {study_nhits.best_trial.params}")

Best value: 0.06079723007900688, Best params: {'input_chunk_length': 164, 'num_stacks': 2, 'num_blocks': 3, 'num_layers': 2, 'layer_exp': 7, 'dropout': 0.060000000000000005, 'lr': 0.00026200382993823117}


In [18]:
plot_optimization_history(study_nhits)

In [19]:
plot_param_importances(study_nhits)

In [20]:
plot_contour(study_nhits, params=["lr", "num_stacks"])

In [21]:
nhits_params = study_nhits.best_params

NHiTS_Model = build_fit_nhits_model(**nhits_params)

training_data = [ts[:-16] for ts in training_transformed] 
y_preds = NHiTS_Model.predict(series=training_data, past_covariates=NHiTS_covariates, n=val_len)
forecasts_back = train_pipeline.inverse_transform(y_preds, partial = True)

for n in range(0,len(forecasts_back)):
    if (list_of_TS[n][:-16].univariate_values()[-14:] == 0).all():
        forecasts_back[n] = forecasts_back[n].map(lambda x: x * 0)

        final_loss = rmsle(list_of_TS, y_preds, inter_reduction = np.mean)

NHiTS_rmsle = rmsle(actual_series = list_of_TS,
                 pred_series = forecasts_back,
                 n_jobs = -1,
                 inter_reduction=np.mean)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [22]:
forecasts_back = train_pipeline.inverse_transform(y_preds, partial = True)

for n in range(0,len(forecasts_back)):
    if (list_of_TS[n][:-16].univariate_values()[-14:] == 0).all():
        forecasts_back[n] = forecasts_back[n].map(lambda x: x * 0)

        final_loss = rmsle(list_of_TS, y_preds, inter_reduction = np.mean)

In [23]:
print("The mean RMSLE for the Global N-HiTS Model over 1782 series is {:.5f}.".format(NHiTS_rmsle))

The mean RMSLE for the Global N-HiTS Model over 1782 series is 0.39741.


In [24]:
# Data Preparation for TFT

def flatten(l):
    return [item for sublist in l for item in sublist]

future_covariates_full = []

for family in family_list:
    future_covariates_full.append(future_covariates_dict[family])
    
future_covariates_full = flatten(future_covariates_full)

# Shift future covariates back so they can be used as past covariates

only_past_covariates = []

for family in family_list:
    only_past_covariates.append(only_past_covariates_dict[family])
    
only_past_covariates = flatten(only_past_covariates)

tft_covariates = []

for i in range(0,len(future_covariates_full)):
    shifted = future_covariates_full[i].shift(n=-16)
    cut = shifted.slice_intersect(only_past_covariates[i])
    stacked = cut.stack(only_past_covariates[i])
    tft_covariates.append(stacked)
    
# Split in train/val/test for Tuning and Validation

val_len = 16

train = [s[: -(2 * val_len)] for s in training_transformed]
val = [s[-(2 * val_len) : -val_len] for s in training_transformed]
test = [s[-val_len:] for s in training_transformed]

In [25]:
def build_fit_tft_model(
    input_chunk_length,
    output_chunk_length,
    hidden_size,
    lstm_layers,
    num_attention_heads,
    full_attention,
    hidden_continuous_size,
    dropout,
    lr,
    likelihood=None,
    callbacks=None,
):
    MAX_N_EPOCHS = 100
    MAX_SAMPLES_PER_TS = 7
    
    early_stopper = EarlyStopping(monitor = "val_loss", min_delta = 1e-4, 
                                  patience = 2, verbose = True)
    
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks
    
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0], 
            "callbacks": callbacks,
        }
        
    model = TFTModel(n_epochs = MAX_N_EPOCHS, input_chunk_length = input_chunk_length, 
                     output_chunk_length = output_chunk_length, hidden_size = hidden_size, 
                     lstm_layers = lstm_layers, num_attention_heads = num_attention_heads, 
                     full_attention = full_attention, hidden_continuous_size = hidden_continuous_size, 
                     dropout = dropout, add_encoders = None, likelihood = None, 
                     batch_size = 128, random_state = 1126, optimizer_kwargs = {"lr": lr}, 
                     loss_fn = torch.nn.MSELoss(), pl_trainer_kwargs=pl_trainer_kwargs, 
                     model_name = "tft_model", force_reset = True, save_checkpoints = True, 
                     add_relative_index = True
    )
    
    model_val_set = [s[-((2 * val_len) + input_chunk_length) : -val_len] for s in training_transformed]
    
    model.fit(series = train, val_series = model_val_set, 
              past_covariates = tft_covariates,
              val_past_covariates = tft_covariates, 
              max_samples_per_ts = MAX_SAMPLES_PER_TS)
    model = TFTModel.load_from_checkpoint("tft_model")

    return model

In [26]:
tft_params = {'input_chunk_length': 230, 
              'output_chunk_length': 16, 
              'hidden_size': 16, 
              'lstm_layers': 3, 
              'num_attention_heads': 4, 
              'full_attention': True, 
              'hidden_continuous_size': 16, 
              'dropout': 0.06, 
              'lr': 0.01}

torch.cuda.empty_cache()

# get the start time
st = time.time()

TFT_Model = build_fit_tft_model(**tft_params)

# Generate Forecasts for the Test Data
training_data = [ts[:-16] for ts in training_transformed] 
preds = TFT_Model.predict(series=training_data, past_covariates=tft_covariates, n=val_len)

# Transform Back
forecasts_back = train_pipeline.inverse_transform(preds, partial=True)

# Zero Forecasting
for n in range(0,len(forecasts_back)):
    if (list_of_TS[n][:-16].univariate_values()[-14:] == 0).all():
        forecasts_back[n] = forecasts_back[n].map(lambda x: x * 0)

# get the end time
et = time.time()

# get the execution time
elapsed_time_tft = et - st

# Mean RMSLE

TFT_rmsle = rmsle(actual_series = list_of_TS,
                 pred_series = forecasts_back,
                 n_jobs = -1,
                 inter_reduction=np.mean)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [27]:
print("The mean RMSLE for the Global TFT Model over 1782 series is {:.5f}.".format(TFT_rmsle))

The mean RMSLE for the Global TFT Model over 1782 series is 0.41027.


In [28]:
# Data Preparation for LSTM

def flatten(l):
    return [item for sublist in l for item in sublist]

future_covariates_full = []

for family in family_list:
    future_covariates_full.append(future_covariates_dict[family])
    
future_covariates_full = flatten(future_covariates_full)

# Shift past covariates forward so they can be used as future covariates


only_past_covariates = []

for family in family_list:
    only_past_covariates.append(only_past_covariates_dict[family])
    
only_past_covariates = flatten(only_past_covariates)


LSTM_covariates = []

for i in range(0,len(only_past_covariates)):
    shifted = only_past_covariates[i].shift(n=16)
    cut = future_covariates_full[i].slice_intersect(shifted)
    stacked = cut.stack(shifted)
    LSTM_covariates.append(stacked)
    
# Slice-Intersect target and covariates after shifting

LSTM_target = []

for i in range(0, len(training_transformed)):
    sliced = training_transformed[i].slice_intersect(LSTM_covariates[i])
    LSTM_target.append(sliced)

# Split in train/val/test for Tuning and Validation

val_len = 16

LSTM_train = [s[: -(2 * val_len)] for s in LSTM_target]
LSTM_val = [s[-(2 * val_len) : -val_len] for s in LSTM_target]
LSTM_test = [s[-val_len:] for s in LSTM_target]

In [29]:
def build_fit_lstm_model(
    input_chunk_length,
    hidden_dim,
    n_rnn_layers,
    lr,
    likelihood=None,
    callbacks=None,
):

    MAX_N_EPOCHS = 100
    MAX_SAMPLES_PER_TS = 60

    # throughout training we'll monitor the validation loss for early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.0001, patience=2, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0], 
            "callbacks": callbacks,
        }
        num_workers = 0
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    # build the LSTM model
    model = RNNModel(
        model="LSTM", input_chunk_length=input_chunk_length, hidden_dim=hidden_dim,
        n_rnn_layers=n_rnn_layers, dropout=0, training_length=input_chunk_length + val_len -1,
        n_epochs=MAX_N_EPOCHS, batch_size=128, add_encoders=None, likelihood=None, 
        loss_fn=torch.nn.MSELoss(), random_state=1126, optimizer_kwargs={"lr": lr},
        pl_trainer_kwargs=pl_trainer_kwargs, model_name="lstm_model", force_reset=True,
        save_checkpoints=True,
    )

    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    model_val_set = [s[-((2 * val_len) + input_chunk_length) : -val_len] for s in LSTM_target]


    # train the model
    model.fit(
        series=LSTM_train, val_series=model_val_set, future_covariates=LSTM_covariates,
        val_future_covariates=LSTM_covariates, max_samples_per_ts=MAX_SAMPLES_PER_TS,
        num_loader_workers=num_workers
    )

    # reload best model over course of training
    model = RNNModel.load_from_checkpoint("lstm_model")

    return model

In [30]:
lstm_params = {'input_chunk_length': 131, 
               'hidden_dim': 39, 
               'n_rnn_layers': 3, 
               'lr': 0.0019971227090605087}

torch.cuda.empty_cache()

# get the start time
st = time.time()

LSTM_Model = build_fit_lstm_model(**lstm_params)

# Generate Forecasts for the Test Data
training_data = [ts[:-16] for ts in LSTM_target] 
preds = LSTM_Model.predict(series=training_data, future_covariates=LSTM_covariates, n=val_len)

# Transform Back
forecasts_back = train_pipeline.inverse_transform(preds, partial=True)

# Zero Forecasting
for n in range(0,len(forecasts_back)):
    if (LSTM_target[n][:-16].univariate_values()[-14:] == 0).all():
        forecasts_back[n] = forecasts_back[n].map(lambda x: x * 0)
        

# get the end time
et = time.time()

# get the execution time
elapsed_time_lstm = et - st

# Mean RMSLE

LSTM_rmsle = rmsle(actual_series = list_of_TS,
                 pred_series = forecasts_back,
                 n_jobs = -1,
                 inter_reduction=np.mean)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [31]:
print("The mean RMSLE for the Global LSTM Model over 1782 series is {:.5f}.".format(LSTM_rmsle))

The mean RMSLE for the Global LSTM Model over 1782 series is 0.40007.


In [32]:
print("Mean RMSLE for Global N-HiTS Model: {:.5f}.".format(NHiTS_rmsle))
print("\n")
print("Mean RMSLE for Global LSTM Model: {:.5f}.".format(LSTM_rmsle))
print("\n")
print("Mean RMSLE for Global TFT Model: {:.5f}.".format(TFT_rmsle))

Mean RMSLE for Global N-HiTS Model: 0.39741.


Mean RMSLE for Global LSTM Model: 0.40007.


Mean RMSLE for Global TFT Model: 0.41027.
